In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from nltk.tokenize import word_tokenize

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Input
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.models import Model
tf.__version__


In [ ]:
# Load the data
df = pd.read_csv('../input/spam-filter/emails.csv')
df.head()

In [ ]:
# checking the number of duplicate columns
df[df.duplicated(subset=None, keep='first') == True].shape

In [ ]:
# dropping the duplicate columns
df.drop_duplicates(inplace = True)

In [ ]:
#1.Punctuations are [!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]
#2.Stop words in natural language processing, are useless words (data).

def process_text(text):
    
    #1 Remove Punctuationa
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    #2 Remove Stop Words
    clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    
    #3 Return a list of clean words
    return clean_words


In [ ]:
#Show the Tokenization (a list of tokens )
df['text'].head().apply(process_text)

In [ ]:
df.text.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
X = df.text
Y = df.spam

In [ ]:
vectorizer = TfidfVectorizer().fit(X)
tfidf_vector_X = vectorizer.transform(X).toarray()
# tfidf_vector_X = tfidf_vector_X[:, :, None]

In [ ]:
tfidf_vector_X[0]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf_vector_X, Y, test_size = 0.2, random_state = 1)

In [ ]:
X_train.shape

In [ ]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
model = tf.keras.Sequential()
model.add(layers.LSTM(64,return_sequences = True, input_shape=(X_train.shape[1] , 1),dropout=0.5))
model.add(layers.Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

# expected shape=(None, None, 200), found shape=(None, 37303, 1)



model.summary()

In [ ]:
# from keras import Sequential
# from keras.layers import LSTM

# model_1 = Sequential()
# model_1.add(LSTM(64,return_sequences = True, input_shape = (4556, 1) ))
# model_1.add(Dense(1,activation='sigmoid'))
# model_1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# print(model_1.summary())



# # model.add(LSTM(units=6, return_sequences=True))
# # model.add(LSTM(units=6, return_sequences=True))
# # model.add(LSTM(units=1, return_sequences=True, name='output'))
# # model.compile(loss='cosine_proximity', optimizer='sgd', metrics = ['accuracy'])

# # print(model.summary())

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=64)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

scores = model.evaluate(X_test, y_test, verbose=0)
y_pred = model.predict_classes(X_test)

print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


In [ ]:
print('confusion matrix:\n', confusion_matrix(y_pred,y_test))